# Edits

* Fixed a factor of 2 in nsqr in the noisevector creator function.
* Added a prolongation factor for the noise vector creator so that a longer noise vector is made that the noise in the strain is then sampled from, to accomodate a short strain (to give the noise the correct properties)
* Added a Tukey window to reduce spectral noise in FFT of strain/signal
* Solved race condition in parallel code
* Changed coalescence time parameter to better align with GW150914 observation length
* Made code to set a chirpmass variable and to refer to this general variable later instead of 30. so that one can run for several events by adding if loops setting the event-dependent parameters.
* Made maxmuq and dmuq variable as well, fixed from the source parameters dependent on the EVENT chosen.
* Changed fmin to 30 Hz which seems to have been used by LIGO/Virgo (See GWTC-1)
* Added save/load-to/from-file for chirp mass data in order for easier smaller chunk Monte Carlo
* Removed parts of code that was not mentioned in paper and was not maintained

## Second edits:
* Made phas array global
* Made all data being saved (each individual run separately) so that one can include the spread of the estimated parameters
* Did the cutoff-dependency at the bottom of file more carefully (consistent grids with the above)
* Minor adjustments


# Matching Uncharged Templates to Charged Mock Data through Matched Filtering

Based on code provided at LIGO open science center and theory found in Maggiore's Gravitational Waves, volume 1

In [ ]:
# Standard python numerical analysis imports:
import numpy as np
from numpy.fft import fft,ifft,fftfreq
from numpy.random import normal
from numpy import diff, pi, sin, sqrt, conj
#from pynverse import inversefunc
from scipy import signal
from scipy.optimize import fsolve
from scipy.stats import linregress
from functools import reduce
#from time import time,clock
import matplotlib.mlab as mlab
import multiprocessing as mp
import os
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

#Fortran made functions (f2py)
#from fortfuncs import makeallparsfort, findsnrfort, getsnrgridfort, findrelationfort

## 0: Units

We'll in the following use $G= k_e = c = M_\odot = 1$, so that everything will be dimensionless, and we have the following conversion factors:

To time, multiply $ \frac{G M_\odot}{c^3}$.

To mass, multiply $ M_\odot$.

To length, multiply $ \frac{G M_\odot}{c^2} $.

## 0.1: Perturbation domain

We will perform the analysis for charges up until our smallness parameter$ \left[ \epsilon \omega^{-2/3} \right]_{\text{max}} = 0.1$.
This implies that 
$$
\left[ \mu \left( \Delta\sigma\right)^2\right]_{\text{max}} = \frac{ 6 \mathcal{M}^{5/4}}{25} \left( \frac{15}{2\tau}\right)^{1/4},
$$
and we see that the linearisation scheme allows for larger charges the smaller observing time we have (paradoxically with the early-inspiral assumption).

In [ ]:
#SWITCH
FORT = False
#NB Fortran version not finished and discontinued

G = 6.67408e-11
c = 2.99792458e8
Ms = 1.989e30
#Conversion factors
Time = G*Ms/c**3.
Length = G*Ms/c**2.

#Parameter vector
getpar = lambda tau0,Mc,muq,phase: np.array([tau0,Mc,muq,phase])
#maxmuq
getmaxmuq = lambda tau0,chirpmass: 6.*chirpmass**(5./4.)*(15./(2.*tau0))**(1./4.)/25.


#More stuff (global)
EVENT='LargeTau0'

if EVENT=='GW151226':
    tau0 = 1.7/Time #Convert from seconds to dimensionless units
    chirpmass = 8.9
    cutoff = 0.01
    phas = np.arange(0.,-pi/5.,-pi/360.)
    
elif EVENT=='GW150914':
    tau0 = 0.2/Time
    chirpmass=28.6
    cutoff = 0.2
    phas = np.arange(0.,-pi/50.,-pi/3000.)
    
elif EVENT=='LargeTau0':
    tau0 = 20./Time
    chirpmass = 30.
    cutoff = 0.002
    phas = np.arange(0.,-2.*pi/3.,-pi/2048.)

elif EVENT=='SmallChirp':
    tau0 = 20./Time
    chirpmass = 3.
    cutoff = 0.0002
    phas = np.arange(0.,-2.*pi,-pi/2048.)
        

maxmuq = getmaxmuq(tau0,chirpmass)
dmuq = maxmuq/7.
    
N = 1. #N is equivalent to sqrt(S0) for the spectral function, adjust to change relative signal and noise amplitudes
fmin = 30.*Time #Detectors sensitivity band starting at 30Hz [GWTC-1]
fmax = 2e3*Time 
f0 = 215.*Time 
fs = 4096.*Time # From LOSC
du = (1./fs)/(tau0)
#From LOSC:
NFFT = int(4*fs/Time)
psd_window = np.blackman(NFFT)
NOVL = int(NFFT/2)

print("Event: ", EVENT,"\ndu: ",du,"\nmaxmuq: ",maxmuq," M_sun\ntau0: ",tau0*Time,"s\nChirp mass: ",chirpmass, " M_sun",sep='')


## 1: Parameters/Symbols

tau0 $\sim\tau_0$ is the initial time to coalescence. We're not interesting in varying this, so we'll assume that it can be fixed and do the analysis below for a fixed value.

Mc $\sim\mathcal{M}^*_c$ is the generalised chirp mass.

muq $\sim\mu (\Delta \sigma )^2$ is the reduced mass times the difference squared of charge to mass ratio of the 2 components.

N $\sim$ max(noise)/max(h) in the first half of inspiral in mock data

And finally h $\sim h$, is the waveform without the polarisation tensor (which is contracted with the detector tensor and just introduces a rescaling with variables that we won't vary (angles..)).

# 2: Template/Mock Data Production

In [ ]:
#Noise
#f>fs, from 0601072
def SnS0(x):
    return x**(-4.14) - 5.*x**(-2.) + \
    111.*(1.-x**2. + 0.5*x**4.)/(1.+0.5*x**2.)



#Some quantities // Results from analytic work
Aw = lambda Mc: 5./( 96.*Mc**(5./3.) )
w0 = lambda tau0,Mc: ( 3.*Aw(Mc)/(8.*tau0) )**(3./8.) #Only to first order
epsw = lambda Mc, muq: 5.*muq/(48.*Mc**(5./3.))
C = lambda tau0, Mc, muq: epsw(Mc,muq) * (3./14.) * w0(tau0,Mc)**(-2./3.)
w_func = lambda u,pars: w0(pars[0],pars[1]) * u**(-3./8.) \
    * (1. - (3./10.) * epsw(pars[1],pars[2])*w0(pars[0],pars[1])**(-2./3.) * u**(1./4.) )

#Phase
Phi = lambda u,par: par[3] + (16./5.)*par[0]*w0(par[0],par[1]) \
    * ( 1. - u**(5./8.) - C(par[0],par[1],par[2])*( 1. - u**(7./8.) ) ) 

#Normalised amplitude, because we don't care about distance or inclination
F = lambda u,par: w0(par[0],par[1])**(2./3.)*u**(-1./4.) \
    - epsw(par[1],par[2])/5.

#The waveform
h = lambda u,par: F(u,par)*np.sin(Phi(u,par))
noh = lambda u,par: u*0.

#Analytically found bias from least squares

rconst = lambda c: ( 8.*(1.-c**(15./8.))*(5./13. - c + 8.*c**(13./8.)/13.)/(15.*(1.-c)) \
                  - 2./15. + 8.*c**(15./8.)/15. - 2.*c**(5./2.)/5.) / \
                ( 25./117. - c + 16.*c**(13./8.)/13. - 4.*c**(9./4.)/9. \
                 - (5./13. - c + 8.*c**(13./8.)/13.)**2./(1.-c) )
sconst = lambda c: 1.-(rconst(c)*(5./13. - c + 8.*c**(13./8.)/13.) + 8.*(1.-c**(15./8.))/15.) \
                / (1.-c)

analytbias = lambda pars,c: 1. + 8.*rconst(c)*C(pars[0],pars[1],pars[2])/5.

analytphas = lambda pars,c: - 16.*pars[0]*w0(pars[0],pars[1]) \
    * sconst(c)*C(pars[0],pars[1],pars[2])/5.

cs = np.linspace(0.,0.4)
plt.plot(cs,rconst(cs)/rconst(0.),label='$r(c)/r_0$')
plt.plot(cs,sconst(cs)/sconst(0.),label='$s(c)/s_0$')
plt.xlabel('c')
plt.legend()
plt.grid()
print(rconst(0))


In [ ]:
def makenoisevec(Npoints):
    dt = 1./fs
    N = int(Npoints/2.)
    x = np.linspace(fmin/f0,fmax/f0,N)
    df = f0*(x[1]-x[0])
    nullen = int(fmin/df)
    NN = N*2 + 2*nullen + 1
    z1 = np.zeros(N)
    z2 = np.zeros(N)
    for i in range(N):
        z1[i] = normal(0,1)
        z2[i] = normal(0,1)
        
    nsqr = N*dt*SnS0(x)
    noise = np.zeros(N,np.complex)
        
    for i in range(N):
        noise[i] = (z1[i]+1.j*z2[i])*sqrt(nsqr[i]/2.)
        
    #NB! Rearrange negative and positive frequencies into convention
    #So that inverse Fourier is real within machine prec.
    f = np.linspace(-fmax,fmax,NN)
    noises = np.zeros(len(f),dtype=np.complex)
    noises[nullen+1:nullen+1+N] = noise
    noises[nullen+N+1:nullen+2*N+1] = conj(noise[::-1])
    return ifft(noises)[:Npoints].real*fs

#Find the power vector
us = np.arange(1.,0.,-du)
prolongationfac = int(20./Time / tau0)
print("Noise production prolongation factor:",prolongationfac)
noisevec = makenoisevec(2*prolongationfac*len(us)) #Multiply with ten to get a better noise sample
sfreq = fftfreq(len(us)*2)*fs
print(len(noisevec), len(us), len(sfreq))
data_psd, freqs = mlab.psd(noisevec, Fs = fs, NFFT = NFFT , window=psd_window, noverlap=NOVL)
power_vec = np.interp(np.abs(sfreq), freqs, data_psd)
df = abs(sfreq[1]-sfreq[0])*fs
plt.plot(noisevec)
plt.show()
#plt.loglog(freqs,data_psd)
plt.loglog(abs(sfreq)[:],power_vec,'--',label='power_vec')
x = np.linspace(fmin/f0,fmax/f0,1000)
plt.loglog(x*f0,SnS0(x),lw=3,label='analytical')
plt.xlim(6e-5,2e-2)
plt.ylim(1,3e4)
plt.grid()
plt.ylabel('$S_n$')
plt.title('Noise spectral density')
plt.legend()

def mock(h,du,N,truepars,cutoff):   
    us = np.arange(1.,0.,-du)
    ht = np.zeros(2*len(us))
    ind = int(len(us)*(1-cutoff))-1
    norm = 1.
    ht[:ind] = h(us[:ind],truepars)
    if (h!=noh):
        #hf = fft(ht)/fs
        #norm = np.sqrt(abs(2.*df*(hf*conj(hf)/power_vec).sum()))
        norm = F(0.05/(Time*truepars[0]),truepars)
    ht = ht/norm
    if (N>0.):
        noisevec = makenoisevec(len(ht))[:len(ht)]
        ht += noisevec*N
    return ht

## Test

In [ ]:
cutstore = cutoff
cutoff=0.
Nstore = N
N= N#0.07
truepars = getpar(tau0,chirpmass,0.,0.)


s = mock(h,du,0.,truepars,cutoff)
us = np.linspace(1.,0.,len(s)//2)
us2 = np.linspace(1,-1,2*len(us))

if EVENT=='GW151226':
    start = int(len(us)/1.1)
    stop = int(len(us2)*0.501)

else:
    start = int(len(us2)*0.45)
    stop = int(len(us2)*0.501)

fig = plt.figure()

ax = fig.add_subplot(223)
plt.plot(us2[start:stop],s[start:stop])
#plt.axvline(x=cutoff,color='r',alpha=0.8)
ax.invert_xaxis()
ax.set_title('Signal')
plt.xlabel('u')
plt.ylabel('s')
ax.yaxis.set_visible(False)


start = int(len(us)/1.2)
stop = len(us2) - int(len(us)*0.9)


ax2 = fig.add_subplot(211)
s = mock(h,du,N,truepars,cutoff)
plt.plot(us2[start:stop],s[start:stop])
ax2.invert_xaxis()
ax2.set_title('Strain')
plt.xlabel('u')
plt.ylabel('s')
ax2.yaxis.set_visible(False)


start = int(len(us)/1.2)
stop = int(len(us2)*0.51)

pars = np.array([-tau0,30.,0.,0.])
s = mock(noh,du,N,pars,cutoff)
ax3 = fig.add_subplot(224)
plt.plot(us2[start:stop],s[start:stop])
ax3.set_title('Noise')
ax3.invert_xaxis()
plt.xlabel('u')
#plt.ylabel('s')
ax3.yaxis.set_visible(False)

fig.tight_layout()
#plt.savefig('waveformAndNoise.eps',bbox_inches='tight')
cutoff = cutstore
N = Nstore

# 3: Finding SNR

In [ ]:
#Following function taken and adapted from LOSC
def findSNR(st,ht,cutoff):
    dwindow =  signal.tukey(st.size, alpha=1./8)
    
    sf = fft(st*dwindow)/fs
    hf = fft(ht*dwindow)/fs    
        
    # -- Calculate the matched filter output in the time domain:
    # Multiply the Fourier Space template and data, and divide by the noise power in each frequency bin.
    SNR = (2.*sf * hf.conjugate()/power_vec).sum()*df
    # -- Normalize the matched filter output: 
    sigmasq = 2.*(hf * hf.conjugate()/power_vec).sum() * df
    sigma = np.sqrt(np.abs(sigmasq))
    #Return a vector (just because of the code at first being written around getting a time-series out)
    return (( SNR/sigma )).real * np.ones(len(sf)) #This is the SNR




# 4: Matched Filtering

In [ ]:
#Make template grid
def makeallpars(tau0,chirps,muqs,phas):
    allpars = np.zeros(len(chirps)*len(muqs)*len(phas)*4).reshape(len(chirps),len(muqs),len(phas),4)
    for i in range(len(chirps)):
        for j in range(len(muqs)):
            for k in range(len(phas)):
                allpars[i,j,k,:] = np.array([tau0,chirps[i],muqs[j],phas[k]])
    return allpars

## Just comparing SNRs

In [ ]:
#The real time-demanding code
#Can't jit because of among other fft
#Huge benefit possible from porting whole thing to Fortran/C then F2Py
def getSNRgrid(allpars,truepars,du,N,cutoff):
    st = mock(h,du,N,truepars,cutoff)
    grid = np.zeros(reduce(lambda x,y: x*y,allpars[:,:,:,0].shape)).reshape(allpars[:,:,:,0].shape)
    for i in range(allpars.shape[0]):
        #print(i/allpars.shape[0],end='\t')
        for k in range(allpars.shape[2]):
            for j in range(allpars.shape[1]):
                temp = findSNR(st,mock(h,du,0.,allpars[i,j,k],cutoff),cutoff)
                grid[i,j,k] = temp.max()
    return grid

def findBestSNR(SNRgrid,allpars):
    maxSNR = SNRgrid.max()
    maxSNRpars = allpars[np.where(SNRgrid==maxSNR)][0]
    return maxSNR,maxSNRpars

## Test

In [ ]:
%%script false --no-raise-error

truepars = getpar(tau0,chirpmass,0.6,0.)

chirps = np.arange(chirpmass-0.1,chirpmass+0.5,0.1) #lowres
muqs = np.array([0.,]) #No charged templates
allpars = makeallpars(tau0,chirps,muqs,phas)

print("Finding SNRs...")
end,start = 0,0
#start = time()
SNRgrid = getSNRgrid(allpars,truepars,du,N,cutoff)
maxSNR,maxSNRpars = findBestSNR(SNRgrid,allpars)
#end = time()

savedtrue = 1.*truepars #Shallow copy
savedfalse = 1.*maxSNRpars


print("Max SNR found:\t",maxSNR,"\nFor parameters (tau0,Mc,muq,phi0):\t",\
      maxSNRpars,"\nTrue parameters:\t\t\t",truepars, \
      "\nTotal time (s):\t",end-start)

And with that we have successfully demonstrated a large mass bias (this is in the generalised chirp mass though..). The above test-code can also be verified to give back the correct parameters when those are covered by the matched filtering search.

Now we turn to look at the relationship between mass bias and $\mu (\Delta \sigma)^2>0$:

# 5: Chirp mass/charge relationship

In [ ]:

def findSNRpar(st,ht,cutoff,muq):
    return [findSNR(st,ht,cutoff),muq]
def getSNRgridpar(allpars,truepars,du,N,cutoff):
    return [getSNRgrid(allpars,truepars,du,N,cutoff),truepars[2]]

includeCharge = False
if includeCharge:
    status = 'withChargeTemplates_'
else:
    status = 'noChargeTemplates_'
    
def findRelation(dMc,dmuq,du,N,runs,cutoff):
    chirps = np.arange(chirpmass-0.2,chirpmass+1.5,dMc)
    muqs = np.arange(0.,.1,1.) #Not contained in matching space
    if includeCharge:
        muqs = np.arange(0.,maxmuq+dmuq,dmuq) #Test whether correctly predicting charge
    allpars = makeallpars(truepars[0],chirps,muqs,phas)
    truqs = np.arange(0.,maxmuq+dmuq,dmuq)
    maxNSNRs = np.zeros(len(truqs)*runs).reshape(runs,len(truqs))
    maxNPars = np.zeros(len(truqs)*4*runs).reshape(runs,len(truqs),4)
    trueSNRs = np.zeros(len(truqs))
    grids = []
    trueres = []
    gridsob = []
    trueresob = []
    truqgrid = np.zeros(len(truqs))
    truqtrur = np.zeros(len(truqs))
    pool = mp.Pool(os.cpu_count())
    #start = time()
    for i in range(len(truqs)):
        #print("\n muqs:\t",round(float(i)/len(truqs),2),end=':\t') #Doesn't work with async
        #print(round(float(j),1)/runs,end='\t')
        gridsob.append((pool.starmap_async(getSNRgridpar,[[allpars,[truepars[0],truepars[1],truqs[i],truepars[3]],du,N,cutoff],]*runs)))
        trueresob.append(pool.starmap_async(findSNRpar,[[mock(h,du,N,[truepars[0],truepars[1],truqs[i],truepars[3]],cutoff),mock(h,du,0.,[truepars[0],truepars[1],truqs[i],truepars[3]],cutoff),cutoff,truepars[2]],]*runs))
        #print('EST Time:\t',(time()-start)*(len(truqs)-i-1)/(i+1)) #Doesn't work with async
    pool.close()
    pool.join
    for i in range(len(truqs)):
        grids.append(gridsob[i].get())
        trueres.append(trueresob[i].get())
        truqgrid[i] = grids[i][0][1]
        truqtrur[i] = trueres[i][0][1]
        for j in range(runs):
            trueSNRs[i] += max(trueres[i][j][0])
            maxNSNRs[j,i],maxNPars[j,i] = findBestSNR(grids[i][j][0],allpars)
   
    #sort
    permutsgrid = np.argsort(truqgrid)
    permutstrur = np.argsort(truqtrur)

    trueSNRs = trueSNRs[permutstrur]/runs
    maxNSNRs = maxNSNRs[:,permutsgrid]
    maxNPars = maxNPars[:,permutsgrid]
    
    #Find the relative SNR
    for i in range(runs):
        maxNSNRs[i] = maxNSNRs[i]/trueSNRs
    
    
    #Save every individual run for finding the spread of the estimated parameters
    
    nums=0
    try:
        infile = open('./chirprelationdata' + EVENT + '_N=' + str(N) + '.bin','rb')
        nums = pickle.load(infile)
        maxNSNRsOld = pickle.load(infile)
        maxNParsOld = pickle.load(infile)
        infile.close()
        totrun = nums+runs
        maxNSNRs2 = np.zeros(len(truqs)*totrun).reshape(totrun,len(truqs))
        maxNPars2 = np.zeros(len(truqs)*4*totrun).reshape(totrun,len(truqs),4)
        maxNSNRs2[:runs] = maxNSNRs[:]
        maxNSNRs2[runs:] = maxNSNRsOld[:]
        maxNPars2[:runs] = maxNPars[:]
        maxNPars2[runs:] = maxNParsOld[:]
        print('Saving together with old Monte Carlo data')
        maxNPars = maxNPars2
        maxNSNRs = maxNSNRs2
        runs = totrun
    except:
        print('No earlier Monte Carlo found\nSaving to new file')

    outfile = open('./chirprelationdata' + EVENT + '_N=' + str(N) + '.bin','wb')
    pickle.dump(runs,outfile)
    pickle.dump(maxNSNRs,outfile)
    pickle.dump(maxNPars,outfile)
    outfile.close()
    
    #Return the averages
    return maxNSNRs.sum(axis=0)/runs, maxNPars.sum(axis=0)/runs, truqs


## Test

In [ ]:
runs = 20


maxSNRs, maxPars, truqs = findRelation(0.01,dmuq,du,N,runs,cutoff)

In [ ]:
#No parallelism beyond here
runInTerminal = True
if runInTerminal:
    quit()

In [ ]:
#Load the data
truqs = np.arange(0.,maxmuq+dmuq,dmuq)
infile = open('./chirprelationdata' + EVENT + '_N=' + str(N) + '.bin','rb')
runs = pickle.load(infile)
maxNSNRs = pickle.load(infile)
maxNPars = pickle.load(infile)
infile.close()

#averages
maxSNRs = maxNSNRs.sum(axis=0)/runs
maxPars = maxNPars.sum(axis=0)/runs

haveSpread = True
if haveSpread:
    #spread
    spreadSNRs = np.zeros(2*len(maxSNRs)).reshape(2,len(maxSNRs))
    spreadPars = np.zeros(2*len(maxPars[0])*len(maxSNRs)).reshape(2,len(maxPars[:,0]),len(maxPars[0,:]))
    
    for i in range(len(maxNSNRs)):
        spreadSNRs[0,:] = maxNSNRs.min(axis=0)
        spreadSNRs[1,:] = maxNSNRs.max(axis=0)
        spreadPars[0,:,:] = maxNPars.min(axis=0)
        spreadPars[1,:,:] = maxNPars.max(axis=0)

print(spreadPars.shape)
print(spreadPars[:,:,1])

In [ ]:
print('Numbers of runs averaged over:',runs)
print("Using cutoff for u<",cutoff,", putting ht=st=0",sep='')
print("Now with tau_0 =",tau0*Time,"s")

maxchirp = np.zeros(len(maxPars))
maxphas = np.zeros(len(maxPars))
maxmuqs = np.zeros(len(maxPars))
for i in range(len(maxSNRs)):
    maxchirp[i] = maxPars[i][1]
    maxphas[i] = maxPars[i][3]
    maxmuqs[i] = maxPars[i][2]
    

fig,ax1 = plt.subplots()


if haveSpread:
    ax1.fill_between(truqs,spreadSNRs[0,:],spreadSNRs[1,:],color='k',alpha=0.4)
plt.plot(truqs,maxSNRs,'--k',lw=1.)
#plt.plot(truqs,maxSNRs2/maxSNRs[0],lw=2., label='2')
#plt.legend()
#plt.grid()
plt.ylabel(r'SNR$_{false}/$SNR$_{true}$')
#plt.xlabel(r'$\mu(\Delta\sigma)^2$')
#plt.legend(loc='best')
plt.grid()
ax1.get_yaxis().get_major_formatter().set_useOffset(False)

ax2 = ax1.twinx()

if haveSpread:
    ax2.fill_between(truqs,spreadPars[0,:,1]/truepars[1],spreadPars[1,:,1]/truepars[1],color='b',alpha=0.4)
    #plt.errorbar(truqs,maxchirp/truepars[1],yerr=spreadPars[:,:,1],fmt='.')
ax2.plot(truqs,maxchirp/truepars[1],'x',ms=9.,label='Matched Filtering')
#plt.plot(truqs,maxPars2[:,1]/30.,lw=2.,label='Matched Filtering2')
#a,b,c,d,e = linregress(truqs,maxPars[:,1]/truepars[1])
#plt.plot(truqs,b+a*truqs)
plt.grid()

plt.ylabel(r'$\mathcal{M}/\mathcal{M}^*_{true}$')
plt.xlabel(r'$\mu (\Delta\sigma)^2/M_\odot$')
ax2.plot(truqs,analytbias([tau0,truepars[1],truqs,1.],cutoff),lw=1.5,label='Least square for phase')
#plt.legend(loc='best')
#plt.title('Averaged over '+ str(runs) + " runs.")
#plt.grid()
#plt.savefig('chirpvscharge.eps',bbox_inches='tight')
#plt.figure()
plt.savefig('chirp_bias_' + status + EVENT + '_N='+str(int(N)) + '_runs=' + str(runs) +'.pdf',bbox_inches='tight')

plt.figure()
plt.title(r'$\Phi_0$')
plt.plot(truqs,maxphas, label='Matched Filtering')
plt.plot(truqs,((analytphas(np.array([truepars[0],truepars[1],truqs,0.]),cutoff))), '--', label='Least square for phase')
plt.ylabel('Radians')
plt.xlabel(r'$\mu (\Delta \sigma)^2$')
plt.grid()
plt.savefig('initial_phase_bias_' + status + EVENT + '_N='+str(int(N)) + '_runs=' + str(runs) +'.eps',bbox_inches='tight')

#plt.legend(loc='best')

plt.figure()
plt.plot(truqs,maxmuqs)
plt.title('charge')
plt.savefig('charge_' + status + EVENT + '_N='+str(int(N)) + '_runs=' + str(runs) +'.eps',bbox_inches='tight')


NB: Code from here on is not adapted to different choices of EVENT. You would then have to adjust the initial phase grids etc manually. Tested for EVENT=LargeTau0

## 6: Cutoff Dependency of SNR & $\mathcal{M}$ (Now comparing charged/uncharged)

In [ ]:

def findRelationCutoff(dMc, muq, cutoffs, du, N, runs):
    chirps = np.arange(chirpmass-chirpmass/150.,chirpmass+chirpmass/30.,chirpmass/300.)
    allpars = makeallpars(truepars[0],chirps,np.array([0,]),phas)

    maxSNRs = np.zeros(len(cutoffs))
    maxChirps = np.zeros(len(cutoffs))
    
    for i in range(len(cutoffs)):
        print("\n cutoff:\t",round(float(i)/len(cutoffs),2),end=':\t')
        for j in range(runs):
            SNRgrid = getSNRgrid(allpars,truepars,du,N,cutoffs[i])
            temp1,temp2 = findBestSNR(SNRgrid,allpars)
            maxSNRs[i] += temp1
            maxChirps[i] += temp2[1]
    return maxChirps/runs

In [ ]:
#Chirp dependence
#start = time()
cutoffs2 = np.linspace(0.,0.4,5)
runs2 = 1
dchirp = 0.1
maxChirps = findRelationCutoff(dchirp,0.6,cutoffs2,du,N,runs2)

In [ ]:
#SNR dependence
truepars = 1.*savedtrue
wrongpars = 1.*savedfalse

cutoffs = np.linspace(0.,0.4,5)
cutSNRs = np.zeros(len(cutoffs))
runs = 10
for i in range(len(cutoffs)):
    temp = 0.
    temp2 = 0.
    st = mock(h,du,N,truepars,cutoffs[i])
    ht = mock(h,du,0.,wrongpars,cutoffs[i])
    for j in range(runs):
        temp += findSNR(st,ht,cutoffs[i])
    cutSNRs[i] = temp.max()/runs
#end = time()
#print("Time (sec):\t",round((end-start),2))


In [ ]:
plt.plot(cutoffs,cutSNRs/cutSNRs[0])
plt.grid()
plt.ylabel(r'$SNR_c/SNR_{c=0}$')
plt.xlabel(r'cutoff, $c$')
plt.title('runs = ' +str(runs) + r',  $\mu\Delta \sigma =$'+str(truepars[2]))
plt.savefig('cutoff'+EVENT+'.png',bbox_inches='tight')
plt.figure()
plt.plot(cutoffs2,maxChirps/truepars[1] - 1,label='numerics')
plt.plot(cutoffs2,8.*C(truepars[0],truepars[1],truepars[2])*rconst(cutoffs2)/5.,label='analytical')
plt.xlabel(r'cutoff, $c$')
plt.ylabel(r'$\mathcal{M}/\mathcal{M}^*_{true}-1$')
plt.title('runs='+str(runs2)+r',  $\mu(\Delta\sigma)^2=0.6$')
plt.legend(loc='best')
plt.savefig('chirpvscutoffswave2'+EVENT+'.png',bbox_inches='tight')
#print(cutoffs2.shape,maxChirps.shape)
print("Lower figure, muq = 0.6")

We see that the cutoff sensitivity is negligible in the LargeTau0 scenario, but that the SNR ratio carries significant effect from the cutoff choice for the two other scenarios.

Found: 

LargeTau0 c$\in(0.,0.2)$:
* Relative SNR difference $\sim2\%$
* Upper relative chirp mass difference $\sim 0.5\%$ (Limited by resolution)

GW151226 c$\in(0.,0.01)$:

* Relative SNR difference $\sim 20\%$
* Upper relative chirp mass difference $1\%$

GW150914 c$\in(0.,0.2)$:
* R SNR diff $\sim 35 \%$
* Upper chirp diff $\sim 3 \%$